In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(lubridate)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
game_sessions <- read_csv("sessions.csv")
game_sessions

In [2]:
game_players <- read_csv("players.csv")
game_players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


In [3]:
glimpse(game_sessions)

ERROR: Error in eval(expr, envir, enclos): object 'game_sessions' not found


In [4]:
player_ss_min <- game_players |>
    select(played_hours, Age) |>
    map_dfr(min, na.rm = TRUE)
player_ss_min

player_ss_max <- game_players |>
    select(played_hours, Age) |>
    map_dfr(max, na.rm = TRUE)
player_ss_max

player_ss_mean <- game_players |>
    select(played_hours, Age) |>
    map_dfr(mean, na.rm = TRUE)
player_ss_mean

player_ss_median <- game_players |>
    select(played_hours, Age) |>
    map_dfr(median, na.rm = TRUE)
player_ss_median

played_hours,Age
<dbl>,<dbl>
0,9


played_hours,Age
<dbl>,<dbl>
223.1,58


played_hours,Age
<dbl>,<dbl>
5.845918,21.13918


played_hours,Age
<dbl>,<dbl>
0.1,19


In [31]:
#knn regression
players_model <- game_players |>
    select(played_hours, Age, gender)|>
    drop_na(played_hours, Age, gender)|>
    mutate(gender = as.factor(gender))
set.seed(2000)

players_split <- initial_split(players_model, prop = 0.75, strata = played_hours)
players_training <- training(players_split)
players_testing <- testing(players_split)

players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
       set_engine("kknn") |>
       set_mode("regression") 

players_recipe <- recipe(played_hours ~ Age + gender, data = players_training) |>
       step_scale(all_numeric_predictors()) |>
       step_center(all_numeric_predictors())

players_vfold <- vfold_cv(players_training, v = 5, strata = played_hours)

players_workflow <- workflow() |>
       add_recipe(players_recipe) |>
       add_model(players_spec)

gridvals <- tibble(neighbors = seq(from = 1, to = 10, by = 2))

players_results <- players_workflow |>
    tune_grid(resamples = players_vfold, grid = gridvals) |>
    collect_metrics()

players_min <- players_results |>
    filter(.metric == "rmse") |>
    slice_min(mean, n = 1)

k_min <- players_min|>
    pull(neighbors)

players_best_spec <- nearest_neighbor(weight_func = "rectangular", 
                                       neighbors = k_min)|>
                        set_engine("kknn")|>
                        set_mode("regression")

players_best_fit <- workflow() |>
                        add_recipe(players_recipe) |>
                        add_model(players_best_spec)|>
                        fit(data = players_training)

players_summary <- players_best_fit |>
                    predict(new_data = players_testing) |>
                    bind_cols(players_testing)|>
                    metrics(truth = played_hours, estimate = .pred)
players_summary

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,56.522693654
rsq,standard,0.002305666
mae,standard,18.410204082


In [33]:
#linear regression
set.seed(2000)
lm_spec <- linear_reg() |>
                set_engine("lm") |>
                set_mode("regression")

session_recipe <- recipe(played_hours ~ Age + gender, data = players_training)

session_fit <- workflow() |>
                add_recipe(session_recipe) |>
                add_model(lm_spec) |>
                fit(data = players_training)

lm_rmse <- session_fit |>
                predict(players_training) |>
                bind_cols(players_training) |>
                metrics(truth = played_hours, estimate = .pred) |>
                filter(.metric == 'rmse') |>
                select(.estimate) |>
                pull()
lm_rmse

lm_rmspe <- session_fit |>
                predict(players_testing) |>
                bind_cols(players_testing) |>
                metrics(truth = played_hours, estimate = .pred) |>
                filter(.metric == 'rmse') |>
                select(.estimate) |>
                pull()
lm_rmspe


[1] 15.888

[1] 49.62139